In [85]:
import json
import os
import numpy as np
metadata_file = 'dataset/slices.geojson'
train_folder = 'dataset/splits/train/'
dev_folder = 'dataset/splits/dev/'
test_folder = 'dataset/splits/test/'

with open(metadata_file) as f:
    d = json.load(f)
patches = d['features']
patches = [{
    **p['properties'], 
     'lng':np.mean(p['geometry']['coordinates'][0][:-1], axis=0)[0],
     'lat':np.mean(p['geometry']['coordinates'][0][:-1], axis=0)[1]
    } for p in patches]
patches[:2]

[{'img_source': '/datadrive/glaciers/unique_tiles/LE07_149037_20041024.tif',
  'mask_source': '/datadrive/glaciers/processed/masks/mask_00.npy',
  'img_slice': '/datadrive/glaciers/processed/slices/slice_0_img_000.npy',
  'mask_slice': '/datadrive/glaciers/processed/slices/slice_0_mask_000.npy',
  'mask_mean_0': 0.0,
  'mask_mean_1': 0.0,
  'mask_mean_2': 0.0,
  'img_mean': 77.38898468017578,
  'lng': 333795.9649122807,
  'lat': 3572176.06391926},
 {'img_source': '/datadrive/glaciers/unique_tiles/LE07_149037_20041024.tif',
  'mask_source': '/datadrive/glaciers/processed/masks/mask_00.npy',
  'img_slice': '/datadrive/glaciers/processed/slices/slice_0_img_001.npy',
  'mask_slice': '/datadrive/glaciers/processed/slices/slice_0_mask_001.npy',
  'mask_mean_0': 0.0,
  'mask_mean_1': 0.0,
  'mask_mean_2': 0.0,
  'img_mean': 133.6044158935547,
  'lng': 348977.8947368421,
  'lat': 3572176.06391926}]

In [86]:
def list_files(folder):
    list_of_files = []

    for root, dirs, files in os.walk(folder):
        for file in files:
            list_of_files.append(file)
    return list_of_files

train_splits = list_files(train_folder)
dev_splits = list_files(dev_folder)
test_splits = list_files(test_folder)

test_splits[:3]

['slice_10_mask_174.npy', 'slice_10_mask_175.npy', 'slice_10_mask_187.npy']

In [87]:
import pandas as pd

metadata = pd.DataFrame(patches)
files_columns = ['img_source','mask_source','img_slice','mask_slice']
metadata[files_columns] = metadata[files_columns].applymap(lambda x: x.split('/')[-1])
metadata['src'] = metadata.img_slice.map(lambda s: s.split('_')[1])
metadata['slice_num'] = metadata.img_slice.map(lambda s: s.split('_')[-1][:3])
metadata.set_index(['src', 'slice_num'], inplace=True)
metadata['mask_mean'] = metadata.mask_mean_0 + metadata.mask_mean_1
metadata.rename(columns={'mask_mean_2':'china'}, inplace=True)
metadata.drop(columns=['img_source', 'mask_source', 'mask_mean_0', 'mask_mean_1'], inplace=True)

metadata.head()

img_slice            mask_slice  china    img_mean  \
src slice_num                                                                 
0   000        slice_0_img_000.npy  slice_0_mask_000.npy    0.0   77.388985   
    001        slice_0_img_001.npy  slice_0_mask_001.npy    0.0  133.604416   
    002        slice_0_img_002.npy  slice_0_mask_002.npy    0.0   94.324028   
    003        slice_0_img_003.npy  slice_0_mask_003.npy    0.0  189.766983   
    004        slice_0_img_004.npy  slice_0_mask_004.npy    0.0  198.229507   

                         lng           lat  mask_mean  
src slice_num                                          
0   000        333795.964912  3.572176e+06        0.0  
    001        348977.894737  3.572176e+06        0.0  
    002        364159.824561  3.572176e+06        0.0  
    003        379341.754386  3.572176e+06        0.0  
    004        394523.684211  3.572176e+06        0.0

In [89]:
def partition_metadata(metadata, splits, folder = ''):
    # takes only the rows corresponding to those in the splits
    res = metadata[metadata.img_slice.isin( splits )].copy()
    # then adds the 
    res.img_slice = folder + res.img_slice
    res.mask_slice = folder + res.mask_slice
    return res

test_data = partition_metadata(metadata, test_splits, folder=test_folder)
dev_data = partition_metadata(metadata, dev_splits, folder=dev_folder)
train_data = partition_metadata(metadata, train_splits, folder=train_folder)

test_data.head()

img_slice  \
src slice_num                                            
1   151        dataset/splits/test/slice_1_img_151.npy   
2   093        dataset/splits/test/slice_2_img_093.npy   
    109        dataset/splits/test/slice_2_img_109.npy   
    157        dataset/splits/test/slice_2_img_157.npy   
3   004        dataset/splits/test/slice_3_img_004.npy   

                                             mask_slice     china    img_mean  \
src slice_num                                                                   
1   151        dataset/splits/test/slice_1_mask_151.npy  0.000000  379.826019   
2   093        dataset/splits/test/slice_2_mask_093.npy  0.001530  525.524292   
    109        dataset/splits/test/slice_2_mask_109.npy  0.020706  510.207916   
    157        dataset/splits/test/slice_2_mask_157.npy  0.031021  537.625977   
3   004        dataset/splits/test/slice_3_mask_004.npy  0.039127  352.693176   

                         lng           lat  mask_mean  
src slice_num                                          
1   151        460217.903596  4.200787e+06   0.521347  
2   093        211011.702409  3.505949e+06   0.316036  
    109        226193.617384  3.521131e+06   0.352470  
    157        271739.362306  3.566677e+06   0.411125  
3   004        654383.836999  3.417196e+06   0.201778